In [1]:
import geopandas as gpd

c:\Users\groen_fe\.conda\envs\ra2ce_env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


Import the OD geopackage from the shared folder

In [28]:
od_path = r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\od\od_matrix_2021_2023_geocoded_nlrtm_de_removed_duplicates.gpkg"
ods = gpd.read_file(od_path)

In [29]:
ods.columns

Index(['UNLO_herkomst', 'UNLO_bestemming', 'v38_Vervoerd_gewicht_sum',
       'v38_Vervoerd_gewicht_count', 'v30_4_Containers_TEU_S_sum',
       'v30_4_Containers_TEU_S_count', 'count_sum', 'count_count', 'geometry'],
      dtype='object')

Save the OD-matrix as an CSV.

In [37]:
# Filter out the trips that have no containers
ods = ods.loc[ods["v30_4_Containers_TEU_S_sum"] > 0]
ods["UNLO_herkomst"].unique()

array(['NLRTM'], dtype=object)

In [41]:
ods[['UNLO_herkomst', 'UNLO_bestemming', 'v38_Vervoerd_gewicht_sum',
       'v38_Vervoerd_gewicht_count', 'v30_4_Containers_TEU_S_sum',
       'v30_4_Containers_TEU_S_count', 'count_count']].to_csv(r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\full_graph\od_matrix.csv", index=False)

In [31]:
# Check the unique origins and destinations 
print(ods["UNLO_herkomst"].unique())
print(ods["UNLO_bestemming"].unique())

['NLRTM']
['DEDUI' 'DELUH' 'DEMAI' 'DENSS' 'DEEMM' 'DEMHG' 'DEKEH' 'DEKAE' 'DEDUS'
 'DEFRA' 'DEMAL' 'DEDMG' 'DEWOR' 'DEBON' 'DEWES' 'DEAND' 'DEDTM' 'DEKOB'
 'DEKRE' 'DEGER' 'DEGIG' 'DEESU' 'DECGN' 'DEWOE' 'DETRI' 'DEVRD' 'DEBRE'
 'DERNH' 'DEHCS' 'DEWLR' 'DEGHM' 'DESTR' 'DEDRP' 'DESN2' 'DEBRV' 'DECUX']


The column 'UNLO_bestemming' contains the name of the location that the geometry is of. Take this column to get all of the locations of the destinations.

In [18]:
# Filter out the destinations that are in the Netherlands
ods = ods.loc[(ods["UNLO_bestemming"] != "NLRTM")]
print(f"Destinations left: {len(ods.index)}")

Destinations left: 36


In [20]:
# Create the Destinations for RA2CE
ds_for_race = gpd.GeoDataFrame()
ds_for_race["geometry"] = ods["geometry"]
ds_for_race["id"] = range(1, len(ods.index) + 1)
ds_for_race["name"] = ods["UNLO_bestemming"]
ds_for_race["category"] = "destination"

In [22]:
# Save the file
ds_for_race.to_file(r"C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\full_graph\static\network\destinations.gpkg", driver="GPKG")

ds